In [37]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, Literal
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
import operator

In [38]:
class ChatState(TypedDict):
    messages : Annotated[list[BaseMessage], add_messages]


In [39]:
llm = ChatOpenAI()

In [40]:
def chat_node(state: ChatState):
    messages = state['messages']
    response = llm.invoke(messages)

    return {'messages': [response]}

In [ ]:
checkpointer = MemorySaver()
graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)
graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node',END)

workflow = graph.compile(checkpointer=checkpointer)

In [47]:
initial_state = {
    'messages' : [HumanMessage(content="What is the capital of india")]
}

workflow.invoke(
    initial_state,
    config={"configurable": {"thread_id": "1"}}
)

{'messages': [HumanMessage(content='What is the capital of india', additional_kwargs={}, response_metadata={}, id='074da54f-0e7e-4f49-976a-51fe3f6cb12c'),
  AIMessage(content='The capital of India is New Delhi.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 13, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-DEPQjqFYOjcsFa8sX7G2esYWNSASo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ca6e7-79b9-7c21-831a-65c253e272e8-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 13, 'output_tokens': 8, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_toke

In [54]:
thread_id = '1'
while True:
    user_messages = input('Type here: ')
    print('user', user_messages)
    if user_messages.strip().lower() in ['exit' , 'quit']:
       break

    config = {'configurable': {'thread_id':thread_id}}
    response = workflow.invoke({'messages': [HumanMessage(content=user_messages)]}, config=config)

    print('AI', response['messages'][-1].content)

user my name is bilal
AI Hello Bilal! How can I assist you today?
user what is my anme
AI Your name is Bilal.
user exit
